<a href="https://colab.research.google.com/github/farieu/rag-ementas/blob/main/1_extracao_ementas_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação de Dados das Ementas para RAG

In [1]:
!pip install pymupdf

In [1]:
import fitz
import re
import pandas as pd
import numpy as np
from pathlib import Path

### Extração de Separação em Blocos

Lê o PDF inteiro e devolve em texto contínuo, em seguida quebra no marcador 'DISCIPLINA', lendo cada campo dos blocos de uma ementa de disciplina e devolve um dicionário com campos limpos.

In [2]:
def extrair_texto_pdf(caminho_pdf: str) -> str:
    with fitz.open(caminho_pdf) as doc:
        return "".join(page.get_text("text") for page in doc)


def separar_blocos(texto: str) -> list[str]:
    return re.split(r'\nDISCIPLINA:\s+', texto)[1:]


def extrair_dados_disciplina(bloco: str) -> dict:
    dados: dict[str, any] = {}

    # Extrai o nome da disciplina (tudo antes de 'CÓDIGO:')
    m_nome = re.search(r'^(.*?)\s*CÓDIGO:', bloco, re.DOTALL)
    if m_nome:
        dados["disciplina"] = m_nome.group(1).strip()
    else:
        dados["disciplina"] = bloco.split("\n", 1)[0].strip()

    # Padrões para campos simples
    simples = {
        "codigo":                r'CÓDIGO:\s*(.*?)\s*\n',
        "departamento":          r'DEPARTAMENTO:\s*(.*?)\s*\n',
        "area":                  r'ÁREA:\s*(.*?)\s*\n',
        "carga_horaria_total":   r'CARGA HORÁRIA TOTAL:\s*(.*?)\s*\n',
        "carga_horaria_semanal": r'CARGA HORÁRIA SEMANAL:\s*(.*?)\s*\n',
        "teoricas":              r'TEÓRICAS:\s*(.*?)\s*\n',
        "praticas":              r'PRÁTICAS:\s*(.*?)\s*\n',
        "pre_requisitos":        r'PRÉ-REQUISITOS:\s*(.*?)\s*\n',
        "co_requisitos":         r'CO-REQUISITOS:\s*(.*?)\s*\n',
    }

    # Padrões para campos longos
    longos = {
        "ementa":               r'EMENTA\s*(.*?)\nCONTEÚDOS',
        "conteudo":             r'CONTEÚDOS\s*(.*?)\nBIBLIOGRAFIA',
        "pratica_curricular":   r'PRÁTICA COMO COMPONENTE CURRICULAR.*?\n(.*?)\n',
    }

    # Extrai campos simples
    for campo, padrao in simples.items():
        m = re.search(padrao, bloco, re.DOTALL)
        dados[campo] = m.group(1).strip() if m else None

    # Extrai campos longos
    for campo, padrao in longos.items():
        m = re.search(padrao, bloco, re.DOTALL)
        dados[campo] = m.group(1).strip() if m else None

    # Extrai bibliografia
    bib_sec = re.search(
        r'BIBLIOGRAFIA\s*(.*?)\n(?:Emissão|Data:|Responsável|$)',
        bloco, re.DOTALL
    )
    if bib_sec:
        bib_texto = bib_sec.group(1)
        # Básica
        m_basica = re.search(
            r'Básica[:\s]*(.*?)(?:\n\s*Complementar|$)',
            bib_texto, re.DOTALL | re.IGNORECASE
        )
        lista_basica = []
        if m_basica:
            lista_basica = [
                re.sub(r'^\s*[\d•\-–\.]+\s*', '', l).strip()
                for l in m_basica.group(1).split("\n") if l.strip()
            ]
        # Complementar
        m_comp = re.search(
            r'Complementar[:\s]*(.*)',
            bib_texto, re.DOTALL | re.IGNORECASE
        )
        lista_comp = []
        if m_comp:
            lista_comp = [
                re.sub(r'^\s*[\d•\-–\.]+\s*', '', l).strip()
                for l in m_comp.group(1).split("\n") if l.strip()
            ]
        dados["bibliografia_basica"] = lista_basica or None
        dados["bibliografia_complementar"] = lista_comp or None
    else:
        dados["bibliografia_basica"] = None
        dados["bibliografia_complementar"] = None

    return dados

###Limpeza dos Textos e Processamento do Pipeline

Tratamento geral para remover quebras múltiplas e espaços dobrados.

In [3]:
def limpar_texto(x):
    if isinstance(x, str):
        x = re.sub(r'\s*\n\s*', ' ', x)          # junta linhas
        return re.sub(r'\s{2,}', ' ', x).strip()
    if isinstance(x, list):
        return [limpar_texto(i) for i in x]
    return x

In [4]:
def processar_pdf(caminho_pdf: str):
    """
    Processa o PDF e retorna a lista de disciplinas encontradas.

    :param caminho_pdf: Caminho para o arquivo PDF de entrada.
    :return: Lista de dicionários com dados de cada disciplina.
    """
    texto = extrair_texto_pdf(caminho_pdf)
    blocos = separar_blocos(texto)
    print(f"Disciplinas encontradas: {len(blocos)}")

    disciplinas = [extrair_dados_disciplina(b) for b in blocos]
    disciplinas = [{k: limpar_texto(v) for k, v in d.items()} for d in disciplinas]

    return disciplinas

### Execução

In [5]:
pdf = "ementasObrigatorias.pdf"          # ajuste caminho
disciplinas = processar_pdf(pdf)
df = pd.DataFrame(disciplinas)
display(df.head())

Disciplinas encontradas: 26


,disciplina,codigo,departamento,area,carga_horaria_total,carga_horaria_semanal,teoricas,praticas,pre_requisitos,co_requisitos,ementa,conteudo,pratica_curricular,bibliografia_basica,bibliografia_complementar
0,ÁLGEBRA VETORIAL E LINEAR PARA COMPUTAÇÃO,06418,MATEMÁTICA,Matemática,60 horas,TEÓRICAS: 04 PRÁTICAS: 00,04 PRÁTICAS: 00,00,Nenhum,None,Álgebra vetorial. Álgebra linear. Métodos numé...,1. Espaços vetoriais 2. Bases 3. Produto escal...,None,"[Steinbruch, Alfredo e Winterle, Paulo. Introd...",[ANTON & RORRES. Álgebra Linear com Aplicações...
1,CÁLCULO NI,XXX,MATEMÁTICA,MATEMÁTICA,60 h,4 h,4 h,0h,NENHUM,NENHUM,Funções Reais de uma Variável Real. Limite e C...,1- FUNÇÕES REAIS DE UMA VARIÁVEL REAL 1.1 – Nú...,,"[[1] STEWART, James. Cálculo, vol. 1, Pioneira...","[[1] ANTON, Howard; BIVENS, Irl; DAVIS, Stephe..."
2,INTRODUÇÃO À CIÊNCIA DA COMPUTAÇÃO,14044,DEINFO,INFORMÁTICA,60h,4h TEÓRICAS: 4h,4h,-,Nenhum,Nenhum,Evolução da Ciência da Computação. Conceitos b...,1. Introdução à Ciência da Computação: o que é...,None,"[BROOKSHEAR, J. Glenn. Ciência da Computação: ...","[WEBER, Raul Fernando. Fundamentos de Arquitet..."
3,INTRODUÇÃO À PROGRAMAÇÃO I,14117,DEINFO,INFORMÁTICA,60h,4h TEÓRICAS: 2h,2h,2h,Nenhum,None,Introdução às linguagens de programação algorí...,1. Fundamentos da construção de algoritmos e p...,NA.,"[Mark LUTZ, David ASCHER: Aprendendo Python, B...","[Mark PILGRIM: Mergulhando No Python, ALTA BOO..."
4,MATEMÁTICA DISCRETA I,14203,DEINFO,INFORMÁTICA,60h,4h TEÓRICAS: 4h PRÁTICAS: -,4h PRÁTICAS: -,-,Nenhum,Nenhum,Lógica proposicional. Lógica de predicados de ...,1. Lógica Proposicional e Técnicas de Demonstr...,None,"[GERSTING, J. L. Fundamentos Matemáticos para ...","[ROSEN, K. H. Matemática Discreta e suas Aplic..."


In [6]:
df.isnull().sum()

,0
disciplina,0
codigo,1
departamento,3
area,1
carga_horaria_total,1
carga_horaria_semanal,1
teoricas,1
praticas,1
pre_requisitos,1
co_requisitos,3


In [7]:
df["codigo"] = df["codigo"].fillna("código não informado")
df["departamento"] = df["departamento"].fillna("departamento não informado")
df["area"] = df["area"].fillna("área não informada")
df["carga_horaria_total"] = df["carga_horaria_total"].fillna("carga horária total não informada")
df["carga_horaria_semanal"] = df["carga_horaria_semanal"].fillna("carga horária semanal não informada")
df["teoricas"] = df["teoricas"].fillna("não informado")
df["praticas"] = df["praticas"].fillna("não informado")
df["pre_requisitos"] = df["pre_requisitos"].fillna("nenhum")
df["co_requisitos"] = df["co_requisitos"].fillna("nenhum")
df["ementa"] = df["ementa"].fillna("ementa não informada")
df["conteudo"] = df["conteudo"].fillna("conteúdo não informado")
df["pratica_curricular"] = df["pratica_curricular"].fillna("não se aplica")
df["bibliografia_basica"] = df["bibliografia_basica"].fillna("bibliografia básica não informada")
df["bibliografia_complementar"] = df["bibliografia_complementar"].fillna("bibliografia complementar não informada")

Colunas relevantes: disciplina, código, departamento, área, carga horária tota, teóricas, práticas, pré-requisitos, co requisitos, ementa, conteúdo, bibliografia basica e complementar.

In [8]:
df.isnull().sum()

,0
disciplina,0
codigo,0
departamento,0
area,0
carga_horaria_total,0
carga_horaria_semanal,0
teoricas,0
praticas,0
pre_requisitos,0
co_requisitos,0


In [10]:
df.head()

,disciplina,codigo,departamento,area,carga_horaria_total,carga_horaria_semanal,teoricas,praticas,pre_requisitos,co_requisitos,ementa,conteudo,pratica_curricular,bibliografia_basica,bibliografia_complementar
0,ÁLGEBRA VETORIAL E LINEAR PARA COMPUTAÇÃO,06418,MATEMÁTICA,Matemática,60 horas,TEÓRICAS: 04 PRÁTICAS: 00,04 PRÁTICAS: 00,00,Nenhum,nenhum,Álgebra vetorial. Álgebra linear. Métodos numé...,1. Espaços vetoriais 2. Bases 3. Produto escal...,não se aplica,"[Steinbruch, Alfredo e Winterle, Paulo. Introd...",[ANTON & RORRES. Álgebra Linear com Aplicações...
1,CÁLCULO NI,XXX,MATEMÁTICA,MATEMÁTICA,60 h,4 h,4 h,0h,NENHUM,NENHUM,Funções Reais de uma Variável Real. Limite e C...,1- FUNÇÕES REAIS DE UMA VARIÁVEL REAL 1.1 – Nú...,,"[[1] STEWART, James. Cálculo, vol. 1, Pioneira...","[[1] ANTON, Howard; BIVENS, Irl; DAVIS, Stephe..."
2,INTRODUÇÃO À CIÊNCIA DA COMPUTAÇÃO,14044,DEINFO,INFORMÁTICA,60h,4h TEÓRICAS: 4h,4h,-,Nenhum,Nenhum,Evolução da Ciência da Computação. Conceitos b...,1. Introdução à Ciência da Computação: o que é...,não se aplica,"[BROOKSHEAR, J. Glenn. Ciência da Computação: ...","[WEBER, Raul Fernando. Fundamentos de Arquitet..."
3,INTRODUÇÃO À PROGRAMAÇÃO I,14117,DEINFO,INFORMÁTICA,60h,4h TEÓRICAS: 2h,2h,2h,Nenhum,nenhum,Introdução às linguagens de programação algorí...,1. Fundamentos da construção de algoritmos e p...,NA.,"[Mark LUTZ, David ASCHER: Aprendendo Python, B...","[Mark PILGRIM: Mergulhando No Python, ALTA BOO..."
4,MATEMÁTICA DISCRETA I,14203,DEINFO,INFORMÁTICA,60h,4h TEÓRICAS: 4h PRÁTICAS: -,4h PRÁTICAS: -,-,Nenhum,Nenhum,Lógica proposicional. Lógica de predicados de ...,1. Lógica Proposicional e Técnicas de Demonstr...,não se aplica,"[GERSTING, J. L. Fundamentos Matemáticos para ...","[ROSEN, K. H. Matemática Discreta e suas Aplic..."


In [9]:
df.to_csv('disciplinas.csv', sep='|', index=False, encoding='utf-8')